## 네이버 카페에서 검색어로 링크 수집 (여러 페이지) 

### 네이버 카페에서 검색어로 링크 수집하는 함수 생성

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests
import json
import pandas as pd
import os, tqdm, time

# 네이버 카페 검색 결과 데이터프레임을 반환하는 함수를 생성합니다.
def getCafeDf(searchWord, bgnDate, endDate, page = 1):
    
    # 요청 URL을 설정합니다.
    url = 'https://apis.naver.com/cafe-home-web/cafe-home/v1/search/articles'
    
    # Post 바디를 딕셔너리로 설정합니다.
    body = {
        'exceptMarketArticle': 1,
        'page': page,
        'period': [bgnDate, endDate], 
        'query': searchWord,
        'sortBy': 0
    }
    
    # 요청 헤더를 딕셔너리로 설정합니다
    headers = {
        'content-type': 'application/json;charset=UTF-8',
        'referer': 'https://section.cafe.naver.com/ca-fe/home/search/articles',
        'user-agent': 'Mozilla/5.0 Chrome/99.0.4844.83 Safari/537.36'
    }
    
    # HTTP 요청을 실행합니다. 
    res = requests.post(url = url, json = body, headers = headers)
    
    # HTTP 응답 상태코드가 정상이면 데이터를 수집합니다.
    if res.status_code == 200:
        
        # HTTP 응답 바디를 딕셔너리로 변환합니다. 
        data = json.loads(s = res.text)
        
        # data에서 검색 결과 개수를 totalCnt에 할당합니다.
        totalCnt = data['message']['result']['totalCount']
        
        # totalCnt가 1 이상이면 데이터프레임을 생성합니다.
        if totalCnt >= 1:
            
            # data['result']에서 검색 결과 데이터를 데이터프레임으로 변환합니다.
            df = pd.DataFrame(data = data['message']['result']['searchResultArticle']['searchResult'])
            
            # 필요한 열이름만 선택합니다.
            cols = ['articleid', 'clubid', 'clubname', 'rawWriteDate', 'subject']
            df = df[cols]
            
            # articleid와 clubid를 문자열로 변환합니다.
            df['articleid'] = df['articleid'].astype('str')
            df['clubid'] = df['clubid'].astype('str')
            
            # rawWriteDate를 날짜 데이터로 변환합니다.
            df['rawWriteDate'] = pd.to_datetime(arg = df['rawWriteDate'], format = '%Y%m%d%H%M')
            
            # df를 반환합니다.
            return df
        
        # totalCnt가 0이면 NULL을 반환합니다.
        else:
            
            # None을 반환합니다.
            return None
    
    # HTTP 응답 상태코드가 정상이 아니면 에러 메시지를 반환합니다.
    else:
        
        print('HTTP 응답 상태코드는 200이 아닙니다!')
        

In [ ]:
# 2 페이지로 테스트합니다.
test = getCafeDf(
    searchWord = '스탠바이미', 
    bgnDate = '20220401', 
    endDate = '20220405', 
    page = 2
)

In [ ]:
# test를 확인합니다.
test

### for 반복문 실행

In [ ]:
# 최종 결과를 저장할 빈 데이터프레임을 생성합니다.
cafe = pd.DataFrame()

In [ ]:
# for 반복문을 실행하여 5 페이지까지 수집합니다.
for i in tqdm.tqdm_notebook(range(5)):
    
    # 지정된 페이지의 데이터프레임을 수집합니다.
    df = getCafeDf(
        searchWord = '스탠바이미', 
        bgnDate = '20220401', 
        endDate = '20220405', 
        page = i + 1
    )
    
    # cafe에 df를 추가합니다.
    cafe = pd.concat(objs = [cafe, df])
    
    # 1초간 멈춥니다.
    time.sleep(1)

In [ ]:
# cafe의 처음 5행을 출력합니다.
cafe.head()

In [ ]:
# cafe의 정보를 확인합니다.
cafe.info()

### xlsx 파일로 저장

In [ ]:
# 현재 작업경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# xlsx 파일명을 설정합니다.
fileName = 'Naver Cafe Links.xlsx'

In [ ]:
# xlsx 파일로 저장합니다.
cafe.to_excel(excel_writer = fileName, index = None)

## End of Document